In [1]:
# !pip install transformers
# !pip install pororo

In [2]:
class CFG:
    SEED = 42
    EPOCHS = 10

# Import

In [3]:
import sys
sys.path.append('/Volumes/KHJ/Github/hyuckjinkim/lib-python/torch')

from torch_seed import seed_everything

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch

In [5]:
seed_everything(CFG.SEED)

<br></br>

# Load Data

데이터를 Pandas의 DataFrame 형식으로 불러오고 어떤 구조인지 확인해 봅니다.  

`train_df`의 경우, id, 네 개의 문장, 정답 문장 번호가 포함되어 있습니다.  

In [6]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

train_df.head()

,id,sentence1,sentence2,sentence3,sentence4,label
0,TRAIN_000,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑 인터넷이 더 싼거 알면서도 이것저것...,직원들 정말 싸가지 없네요 ㅋㅋㅋㅋ 인터넷에서 더 싸게 구입할 수 있다는 걸 알면서...,직원들의 태도가 정말 별로였어요 ㅋㅋㅋㅋ 가볼만한 가게라는 소문을 듣고 인터넷으로 ...,2
1,TRAIN_001,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,분위기가 너무 좋아요! 2층 창문이 넓어서 쾌적한 느낌이에요. 조명도 아름답고 음료...,분위기가 짱!! 2층 창문이 커서 탁 트여있는 느낌이에요 ㅎㅎ 조명도 예쁘고 음료랑...,분위기가 너무 좋아요! 2층 창문이 크고 넓어서 탁 트여있는 느낌이에요. 조명도 예...,3
2,TRAIN_002,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",일단 장사가 잘되길 바라는 마음에서 별5개 드립니다 간도 맞았고 매운걸 좋아하는 입...,일단 저는 장사가 잘되기를 바라는 마음에서 별 다섯 개를 주고 싶어요. 맛도 딱 맞...,"먼저, 칭찬과 응원의 의미로 별 다섯 개를 주고 싶습니다. 간도 딱 맞고, 저는 매...",2
3,TRAIN_003,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...","1편의 신선함에 비해 약간 빛이 바래 보이지만, 여전히 재미있게 즐길 수 있어요. ...","1편의 독특함 때문에 약간의 비교가 불가피하지만, 이 게임은 여전히 흥미로워요. 시...",1편이 워낙 참신했던 탓에 좀 묻힌 감이 있긴 하지만 재미는 여전합니다. 시스템도 ...,4
4,TRAIN_004,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",빵점을 주고 싶지만 참아냈습니다... 이 영화는 사상 유래가 없는 것 같아요. 조지...,빵점 주고 싶을 정도로 엄청 실망했어요... 이 영화는 별들의 전쟁처럼 역사적인 작...,"빵점을 주고 싶었는데 참았어요... 이런 영화는 전례가 없는데, 조지 루카스의 스타...",1


In [7]:
train_df.shape, test_df.shape

((50, 6), (1100, 5))

In [8]:
data = train_df.copy()
data.drop('id',axis=1,inplace=True)

new_data = []
for d in data.values:
    sentence = d[:-1]
    label    = d[-1]
    
    human_sentence = sentence[label-1]
    ai_sentence    = sentence[sentence!=human_sentence]
    
    new_d = pd.concat([
        pd.DataFrame([human_sentence],columns=['text']).assign(label=1),
        pd.DataFrame(ai_sentence     ,columns=['text']).assign(label=0),
    ],axis=0)
    new_data.append(new_d)
    
new_data = pd.concat(new_data,axis=0)
new_data.reset_index(drop=True,inplace=True)

<br></br>

# Model

In [9]:
# from pororo import Pororo

In [10]:
from tqdm import tqdm, trange

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import PreTrainedTokenizerFast, GPT2ForSequenceClassification
from transformers import AdamW

/Users/khj/anaconda3/envs/torch_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
class CompDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __getitem__(self, index):
        text  = self.data.text[index]
        label = self.data.label[index]
        
        encoded_dict = self.tokenizer.encode_plus(
            text,
            add_special_tokens = True,
            max_length = 512,
            pad_to_max_length = True,
            truncation = True,
            return_attention_mask = True,
            return_tensors = 'pt',
        )
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]

        return {'input_ids': padded_token_list, 'attention_mask': att_mask, 'labels': label}

    def __len__(self):
        return len(self.data)

In [14]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(new_data,test_size=0.2,shuffle=True,random_state=CFG.SEED)
train_data.reset_index(drop=True,inplace=True)
val_data  .reset_index(drop=True,inplace=True)
print(train_data.shape, val_data.shape)

(160, 2) (40, 2)


In [15]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
    pad_token='<pad>', mask_token='<mask>',
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [16]:
train_dataset = CompDataset(data=train_data,tokenizer=tokenizer)
val_dataset   = CompDataset(val_data  ,tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False, num_workers=0)
val_loader   = DataLoader(val_dataset  , batch_size=8, shuffle=False, num_workers=0)

In [17]:
import torch.nn as nn
class BaseModel(nn.Module):
    def __init__(self, num_classes):
        super(BaseModel, self).__init__()
        
        self.model = GPT2ForSequenceClassification.from_pretrained("skt/kogpt2-base-v2", num_labels=num_classes, return_dict=True, problem_type="single_label_classification")
        # self.model = AutoModel.from_pretrained('skt/kogpt2-base-v2')
        # self.model.fc = nn.Linear(self.model.ln_f.normalized_shape[0], num_classes)
        
    def forward(self, input_ids, attention_mask, labels):
        # inputs = self.tokenizer(text, return_tensors="pt")
        # outputs = self.model(**inputs,return_dict=True)
        # return outputs
        
        outputs = self.model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            labels = labels,
        )
        return outputs

In [18]:
# 현재 GPU가 사용 가능한지 확인합니다.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [19]:
model = BaseModel(num_classes=2)
model.to(device)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BaseModel(
  (model): GPT2ForSequenceClassification(
    (transformer): GPT2Model(
      (wte): Embedding(51200, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (score): Linear(in_features=768, out_features=2, bias=False)
  )
)

In [20]:
criterion = nn.NLLLoss()
optimizer = AdamW(model.parameters(), lr=1e-5)

/Users/khj/anaconda3/envs/torch_env/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
for epoch in trange(CFG.EPOCHS):
    
    model.train()
    train_loss = []
    pbar = tqdm(train_loader,desc='Train Iter',leave=False,position=0)
    for inputs in pbar:
        labels  = inputs['labels']
        outputs = model(**inputs)
        
        #loss = outputs.loss
        log_probs = F.log_softmax(outputs.logits, dim=1)
        loss = criterion(log_probs, labels)
        train_loss.append(loss.item())
        
        loss.backward()
        optimizer.step()
    
    with torch.no_grad():
        val_loss = []
        pbar = tqdm(val_loader,desc='Validation Iter',leave=False,position=0)
        for inputs in pbar:
            labels  = inputs['labels']
            outputs = model(**inputs)
            
            #loss = outputs.loss
            log_probs = F.log_softmax(outputs.logits, dim=1)
            loss = criterion(log_probs, labels)
            val_loss.append(loss.item())
            
    progress = 'loss={:.4f}, val_loss={:.4f}'.format(np.mean(train_loss),np.mean(val_loss))
    print(progress)

Train Iter:   0%|                                        | 0/20 [00:00<?, ?it/s]/Users/khj/anaconda3/envs/torch_env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
 10%|████▎                                      | 1/10 [03:25<30:52, 205.88s/it]

loss=0.8239, val_loss=0.4459


 20%|████████▌                                  | 2/10 [07:10<28:56, 217.12s/it]

loss=1.1821, val_loss=1.1697


 30%|████████████▉                              | 3/10 [10:57<25:50, 221.51s/it]

...

loss=0.4587, val_loss=0.7518


 90%|██████████████████████████████████████▋    | 9/10 [33:23<03:43, 223.97s/it]

loss=0.8616, val_loss=0.4562


100%|██████████████████████████████████████████| 10/10 [37:08<00:00, 222.84s/it]

loss=0.3414, val_loss=0.4443


<br></br>

# Inference

train_df에서 행 하나를 랜덤하게 추출하여 example로 사용합니다(one-shot)  

이때 실제 사람이 작성한 리뷰에 대해서는 '-> O'로 표시하고 그렇지 않은 리뷰에 대해서는 '-> X'로 표기합니다.

In [ ]:
example_sentence_format = """
전달하는 4개의 문장 중에서 1개만 사람이 작성한 것이고, 나머지는 AI가 작성한 것입니다.
주어진 문장이 사람이 작성한 것이 맞으면 O, 아니면 X를 반환하세요.

# 예시는 아래와 같습니다.
문장1 : {} -> {}
문장2 : {} -> {}
문장3 : {} -> {}
문장4 : {} -> {}

# 문제는 아래와 같습니다.
문장 : {}
"""

In [ ]:
%%time

model.eval()

preds = []
with torch.no_grad():
    # 각 테스트 케이스에 대해
    for idx in tqdm(range(len(test_df))):
        row = test_df.iloc[idx]
        best_score = float('-inf')
        best_label = 0
        
        # train 데이터에서 랜덤하게 문장을 가져옵니다.
        random_row = train_df.sample(1).iloc[0]
        random_answer = random_row['label']
        random_labels = {}
        for i in range(1, 5):
            random_labels[f'sentence{i}'] = 'O' if i == random_answer else 'X'

        # 각 문장에 대한 확률값을 구하고, 가장 높은 확률값을 가진 문장을 선택합니다.
        for i in range(1, 5):
            # GPT-2에게 제공할 prompt를 작성합니다.
            prompt = example_sentence_format.format(
                random_row['sentence1'],random_labels['sentence1'],
                random_row['sentence2'],random_labels['sentence2'],
                random_row['sentence3'],random_labels['sentence3'],
                random_row['sentence4'],random_labels['sentence4'],
                row[f'sentence{i}']
            )
            inputs = tokenizer(prompt, return_tensors="pt")
            inputs = inputs.to(device)
            with torch.no_grad():
                outputs = model(**inputs)
                score = outputs[0][:, -1, :].max().item()

            if score > best_score:
                best_score = score
                best_label = i

        preds.append(best_label)
        
        ddasfsd

<br></br>

# Post-processing

본 대회에서는 각 케이스에 대한 예측값을 두 개 제출할 수 있기 때문에 나머지 하나를 채워줍니다.  

베이스라인에서는 1,2,3,4 중에서 4를 예시로 사용했습니다.

In [ ]:
preds = [str(pred) + '4' for pred in preds]
preds[:5]

<br></br>

# Submission

데이터셋에 함께 제공되는 `sample_submission.csv` 파일을 불러와서 `label` 칼럼을 업데이트하고 결과를 제출합니다.

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['label'] = preds
submit.head()

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)